ELEC6910X Advanced Topics in AI and Healthcare

# Assignment 1

***Holy Lovenia - 20814158***

------

In [1]:
from collections import OrderedDict
from PIL import Image
from pytorch3dunet.unet3d import losses
from torch import nn
from torch.autograd import Variable
from torch.nn import init
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomVerticalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)
from transformers import AutoConfig, AutoFeatureExtractor, AutoModelForImageClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from tqdm import tqdm

import datasets
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import surface_distance.metrics as sf
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchio as tio
import torchmetrics
import torchvision

In [2]:
RANDOM_SEED = 42

def set_all_seeds(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True

set_all_seeds(RANDOM_SEED)

## Problem 2

In [3]:
model_name_or_path = "microsoft/resnet-152"
# model_name_or_path = "gianlab/swin-tiny-patch4-window7-224-finetuned-skin-cancer"
# model_name_or_path = "google/vit-base-patch16-224-in21k"

In [4]:
metric = datasets.load_metric("accuracy")

/tmp/ipykernel_31519/3770023701.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("accuracy")


In [5]:
def problem2_load_dataset(manifest_path, img_dir_path, img_column_name="image_train", label_column_name="label_train"):
    manifest_df = pd.read_csv(manifest_path)
    manifest_df[img_column_name] = img_dir_path + manifest_df[img_column_name].astype(str)
    manifest_df = manifest_df.rename(columns={img_column_name: "image", label_column_name: "label"})
    dataset = datasets.Dataset.from_pandas(manifest_df)
    dataset = dataset.cast_column("image", datasets.Image(decode=True))
    dataset = dataset.class_encode_column("label")
    return dataset

In [6]:
def ham10000_load_dataset(manifest_path, img_dir_paths, img_column_name="image_id", label_column_name="dx"):
    manifest_df = pd.read_csv(manifest_path)
    for i in range(len(manifest_df)):
        for img_dir_path in img_dir_paths:
            img_path = os.path.join(img_dir_path, f"{manifest_df[img_column_name][i]}.jpg")
            if os.path.isfile(img_path):
                manifest_df[img_column_name][i] = img_path
    manifest_df = manifest_df.drop(manifest_df[~manifest_df[img_column_name].str.contains(".jpg")].index)
    manifest_df = manifest_df.rename(columns={img_column_name: "image", label_column_name: "label"})
    manifest_df = manifest_df.replace({
        "mel": 0,
        "nv": 1,
        "bcc": 2,
        "akiec": 3,
        "bkl": 4,
        "df": 5,
        "vasc": 6,
    })
    dataset = datasets.Dataset.from_pandas(manifest_df)
    dataset = dataset.cast_column("image", datasets.Image(decode=True))
    dataset = dataset.class_encode_column("label")
    return dataset

In [7]:
raw_datasets = datasets.DatasetDict()
raw_datasets["train"] = problem2_load_dataset("./data/problem2_datas/annotation/train.csv", "./data/problem2_datas/images/")
# train_val_dataset = problem2_load_dataset("./data/problem2_datas/annotation/train.csv", "./data/problem2_datas/images/").train_test_split(test_size=0.2)
# raw_datasets["train"] = train_val_dataset["train"]
# raw_datasets["valid"] = train_val_dataset["test"]
raw_datasets["test"] = problem2_load_dataset("./data/problem2_datas/annotation/test.csv", "./data/problem2_datas/images/")

Stringifying the column:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Stringifying the column:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
pretrain_datasets = datasets.DatasetDict()
pretrain_train_val_dataset = ham10000_load_dataset(
    "./data/HAM10000/HAM10000_metadata.csv",
    [
        "./data/HAM10000/ham10000_images_part_1",
        "./data/HAM10000/HAM10000_images_part_1",
        "./data/HAM10000/ham10000_images_part_2",
        "./data/HAM10000/HAM10000_images_part_2"
    ]).train_test_split(test_size=0.2)
pretrain_datasets["train"] = pretrain_train_val_dataset["train"]
pretrain_datasets["valid"] = pretrain_train_val_dataset["test"]

/tmp/ipykernel_31519/4003405763.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manifest_df[img_column_name][i] = img_path


Stringifying the column:   0%|          | 0/11 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/11 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

In [9]:
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)
model = AutoModelForImageClassification.from_pretrained(model_name_or_path)

In [10]:
normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
train_transforms = Compose(
        [
#             RandomResizedCrop(feature_extractor.size),
            Resize(feature_extractor.size),
            CenterCrop(feature_extractor.size),
            RandomHorizontalFlip(),
            RandomVerticalFlip(),
            ToTensor(),
            normalize,
        ]
    )

eval_transforms = Compose(
        [
            Resize(feature_extractor.size),
            CenterCrop(feature_extractor.size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_eval(example_batch):
    """Apply eval_transforms across a batch."""
    example_batch["pixel_values"] = [
        eval_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

In [11]:
proc_datasets = raw_datasets
proc_datasets["train"].set_transform(preprocess_train)
# proc_datasets["valid"].set_transform(preprocess_eval)
proc_datasets["test"].set_transform(preprocess_eval)

In [12]:
proc_pretrain_datasets = pretrain_datasets
proc_pretrain_datasets["train"].set_transform(preprocess_train)
proc_pretrain_datasets["valid"].set_transform(preprocess_eval)

In [13]:
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [14]:
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

#### Pretrain

In [ ]:
args = TrainingArguments(
    f"save/problem2/pretrain__{model_name_or_path}__1e-4",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=200,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    gradient_checkpointing=True,
    save_total_limit=1,
    seed=RANDOM_SEED,
    data_seed=RANDOM_SEED,
    dataloader_num_workers=4,
    lr_scheduler_type="cosine",
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=proc_pretrain_datasets["train"],
    eval_dataset=proc_pretrain_datasets["valid"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)

In [ ]:
train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

In [ ]:
metrics = trainer.evaluate(proc_datasets["test"])
# some nice to haves:
trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)

In [ ]:
metrics = trainer.evaluate(proc_pretrain_datasets["valid"])
# some nice to haves:
trainer.log_metrics("valid", metrics)
trainer.save_metrics("valid", metrics)

In [ ]:
metrics = trainer.evaluate(proc_pretrain_datasets["train"])
# some nice to haves:
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)

#### Problem 2 dataset

In [15]:
# model = AutoModelForImageClassification.from_pretrained(f"save/problem2/pretrain__{model_name_or_path}__1e-4")
model = AutoModelForImageClassification.from_pretrained(model_name_or_path)

In [16]:
args = TrainingArguments(
#     f"save/problem2/pretrained__1e-4|{model_name_or_path}__5e-4",
    f"save/problem2/{model_name_or_path}__1e-3__cwr",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=200,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    gradient_checkpointing=True,
    save_total_limit=1,
    seed=RANDOM_SEED,
    data_seed=RANDOM_SEED,
    dataloader_num_workers=4,
    lr_scheduler_type="cosine_with_restarts",
)

In [17]:
class Problem2Trainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to("cuda")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits").to("cuda")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.11, 0.68, 0.05, 0.03, 0.11, 0.01, 0.01] + [0 for i in range(993)])).to("cuda")
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [18]:
trainer = Problem2Trainer(
    model,
    args,
    train_dataset=proc_datasets["train"],
    eval_dataset=proc_datasets["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=20)],
)

In [19]:
train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/home/holy/anaconda3/envs/health/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 305
  Num Epochs = 200
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 400


RuntimeError: CUDA out of memory. Tried to allocate 98.00 MiB (GPU 0; 23.70 GiB total capacity; 1.08 GiB already allocated; 70.56 MiB free; 1.17 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
metrics = trainer.evaluate(proc_datasets["test"])
# some nice to haves:
trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)

In [ ]:
# metrics = trainer.evaluate(proc_datasets["valid"])
# # some nice to haves:
# trainer.log_metrics("valid", metrics)
# trainer.save_metrics("valid", metrics)

In [ ]:
metrics = trainer.evaluate(proc_datasets["train"])
# some nice to haves:
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)